In [1]:
import numpy as np
import random
import time

## Reading and preparing  dataset

In [2]:
import pandas as pd
from pathlib import Path

DATASET_FOLDER = Path('data/the_movies_dataset')

SMALL_DATASET = True

RATINGS_FILEPATH = DATASET_FOLDER / 'ratings_small.csv' if SMALL_DATASET else DATASET_FOLDER / 'ratings.csv'
MOVIES_FILEPATH = DATASET_FOLDER / 'movies_metadata.csv'

In [3]:
ratings = pd.read_csv(RATINGS_FILEPATH)

In [4]:
movies = pd.read_csv(MOVIES_FILEPATH)

C:\Users\subru\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
movies["popularity"] = pd.to_numeric(movies["popularity"], errors ='coerce').fillna(0.0).astype('float')

In [6]:
popular_movies=movies[movies["popularity"]> 3]["id"].astype('int').tolist()

In [7]:
ratings=ratings[ratings["movieId"].isin(popular_movies) ].reset_index(drop=True)

In [8]:
movie_embeddings = ratings.pivot_table(columns='userId',index='movieId',values='rating',fill_value=0.0)

##  Algorithms Implementation

In [9]:
class HashTable:
    def __init__(self, hash_size, inp_dimensions):
        self.hash_size = hash_size
        self.inp_dimensions = inp_dimensions
        self.hash_table = dict()
        self.projections = np.random.randn(self.hash_size, inp_dimensions)
        
    def generate_hash(self, inp_vector):
        bools = (np.dot(inp_vector, self.projections.T) > 0).astype('int')
        return ''.join(bools.astype('str'))
    
    def __setitem__(self, inp_vec, label):
        #print("generating hash value for hash table generation")
        hash_value = self.generate_hash(inp_vec)
        #print("completed generating hash value for hash table generation")
        self.hash_table[hash_value] = self.hash_table.get(hash_value, list()) + [label]
        #print("completed Setting values to hash table")
    def __getitem__(self, inp_vec):
        #print("generating hash value for searching hash table")
        hash_value = self.generate_hash(inp_vec)
        return self.hash_table.get(hash_value, [])
        

In [10]:
class LSH:
    def __init__(self, num_tables, hash_size, inp_dimensions):
        self.num_tables = num_tables
        self.hash_size = hash_size
        self.inp_dimensions = inp_dimensions
        self.hash_tables = list()
        for i in range(self.num_tables):
            self.hash_tables.append(HashTable(self.hash_size, self.inp_dimensions))
            
    def __setitem__(self, inp_vec, label):
        #print("generating hash tables")
        for table in self.hash_tables:
            table[inp_vec] = label
        #print("hash table generation complete")
    
    def __getitem__(self, inp_vec):
        results = list()
        for table in self.hash_tables:
            #print("looping through hash tables")
            results.extend(table[inp_vec])
            #print("suggestions from table ",results)
          
        return list(set(results))
    



In [11]:
nusers=movie_embeddings.columns
nmovies=movie_embeddings.index

In [12]:
hash_table = LSH(num_tables=20,hash_size=10, inp_dimensions=len(nusers))

In [13]:
for i in range(len(nmovies)):
    hash_table[movie_embeddings.loc[nmovies[i]]]=nmovies[i]

In [14]:
# for hashtab  in hash_table.hash_tables:
#     print(hashtab.hash_table)
    

In [15]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [16]:
def getJaccardSim(movie1,movie2):
    movie1=''.join((movie1 > 3).astype(int).astype('str'))
    movie2=''.join((movie2 > 3).astype(int).astype('str'))
    N = 0
    D = 0
    for i in range(len(movie1)):
        sum_ = int(movie1[i]) + int(movie2[i])
        if(sum_ >= 1):
            flag = 1
            D += 1
            if(sum_ == 2):
                N += 1
    if D == 0:
        return 0
    return(float(N)/D)

In [17]:
def getCosineSim(movie1,movie2):

    return np.dot(movie1,movie2)/(np.linalg.norm(movie1)*np.linalg.norm(movie2))

In [18]:
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

In [19]:
def lsh(m,k,sim_measure):
    inp_vec=movie_embeddings.loc[m]
    #print("Movie_id" ,m)
    #print("Query movie name",movies[movies["id"]==str(m)]["original_title"].values[0])
    similar_movies = hash_table[inp_vec]
    
    similarity_values =[]
    for a in similar_movies:
        if a== m:
            continue
        out_vec = movie_embeddings.loc[a]
        similarity=0
        if sim_measure == "cos":
            similarity = getCosineSim (inp_vec,out_vec)
        elif sim_measure == "jac":
            similarity = getJaccardSim (inp_vec,out_vec)
        similarity_values.append(similarity)
        
    ranked_sim = np.argsort(np.array(similarity_values))[::-1][:k]
    movie_sugg=[]
    sim_values=[]
    for i in range (0,k):
        movie_id = similar_movies[ranked_sim[i]]
        movie_sugg.append(movie_id)
        sim_values.append(similarity_values[ranked_sim[i]])
    return movie_sugg,sim_values
    

In [20]:
def knn(m,k):
    embeddings_sparse = csr_matrix(movie_embeddings)

    model = NearestNeighbors(n_neighbors=k,algorithm='brute',metric='cosine')
    model.fit(embeddings_sparse)
    movie_emb = movie_embeddings.loc[m,:]
    distances,suggestions=model.kneighbors(movie_emb.values.reshape(1,-1),k+1)
    suggestions= suggestions.flatten()
    distances= distances.flatten()
    movies = []
    similarity =[]
    for i in range(1,len(suggestions)):
        movies.append(movie_embeddings.index[suggestions[i]])
        similarity.append(1-distances[i])   

    return movies,similarity 
    

## Benchmarking algorithms

In [21]:
def benchmark_rating_pred(test_movies,k,model):
    predicted_ratings=pd.DataFrame(columns=['movie_id', 'predicted_rating','user_id'])
    if model=="lsh_cos":
        sim_indicator="cos"
    elif  model=="lsh_jac":
        sim_indicator="jac"
    for index,m in enumerate(test_movies):
        if model == "knn":
            similar_movies,sim_values = knn(m,k)
        else:
            similar_movies,sim_values=lsh(m,k,sim_indicator)
            #print(similar_movies)
        count_similar=len(similar_movies)
        selected_user=ratings[ratings["movieId"].isin(similar_movies) & ratings["rating"]!=0]["userId"].value_counts().nlargest(1).index[0]
        rating_N=0
        rating_D=0
            #print("selected user",selected_user)
        for idx,i in enumerate(similar_movies):
            rating_val=movie_embeddings.loc[i][selected_user]
            sim_val=sim_values[idx]
                #print("sim val", sim_val,"rating val",rating_val)
            rating_N=rating_N+sim_val*rating_val
                #print("numerator",rating_N)
            rating_D=rating_D+sim_val
                #print("denominator",rating_D)
            #print("final numerator",rating_N,"final denominator",rating_D)
        predicted_ratings.loc[index] = [int(m), float(rating_N/rating_D),int(selected_user)]
    return predicted_ratings
    
    
    

In [22]:
def actual_rating(x):
    y=ratings[(ratings["userId"]==x.user_id) & (ratings["movieId"]==x.movie_id)]["rating"]
    if len(y) != 0:
        return y.values[0]  
    else:
        return 0

In [23]:
def get_rmse(ratings_pred):
    ratings_pred["actual_rating"]=ratings_pred.apply(actual_rating ,axis=1)
    ratings_pred["squared_diff"]=np.square(ratings_pred["actual_rating"]-ratings_pred["predicted_rating"])
    rmse = np.sqrt(ratings_pred["squared_diff"].mean())
    return rmse
    

In [24]:
test_movies=random.sample(list(ratings["movieId"].unique()),10)

## LSH cosine Performance

In [25]:
start_time = time.time()
ratings_pred=benchmark_rating_pred(test_movies,5,"lsh_cos")
exec_time= time.time() - start_time

In [26]:
print("Performance of LSH with cosine rmse : ",get_rmse(ratings_pred),"execution time : ",exec_time)

Performance of LSH with cosine rmse :  1.1635812944570985 execution time :  0.6890490055084229


In [27]:
ratings_pred

,movie_id,predicted_rating,user_id,actual_rating,squared_diff
0,170.0,2.130751,564.0,4.0,3.494091
1,103731.0,1.680904,547.0,0.0,2.825438
2,173.0,4.202584,564.0,4.0,0.041040
3,5820.0,2.808730,509.0,4.0,1.419124
4,2470.0,1.411018,388.0,3.0,2.524865
5,916.0,4.376745,575.0,5.0,0.388447
6,1632.0,2.600000,165.0,2.5,0.010000
7,1956.0,3.771410,452.0,4.0,0.052254
8,7010.0,3.095932,15.0,3.5,0.163271
9,6936.0,2.381147,380.0,4.0,2.620685


## LSH jacard Performance

In [28]:
start_time = time.time()
ratings_pred=benchmark_rating_pred(test_movies,5,"lsh_jac")
exec_time= time.time() - start_time

C:\Users\subru\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in double_scalars


In [29]:
print("Performance of LSH with jacard rmse : ",get_rmse(ratings_pred),"execution time : ",exec_time)

Performance of LSH with jacard rmse :  1.6353172322122065 execution time :  2.1475841999053955


In [30]:
ratings_pred

,movie_id,predicted_rating,user_id,actual_rating,squared_diff
0,170.0,2.103933,564.0,4.0,3.595072
1,103731.0,2.444444,652.0,5.0,6.530864
2,173.0,3.341838,564.0,4.0,0.433177
3,5820.0,2.214286,509.0,4.0,3.188776
4,2470.0,1.788478,73.0,3.0,1.467786
5,916.0,1.963689,311.0,4.5,6.432872
6,1632.0,NaN,564.0,0.0,NaN
7,1956.0,4.426419,575.0,4.0,0.181833
8,7010.0,3.447368,15.0,3.5,0.002770
9,6936.0,3.495062,73.0,2.0,2.235212


## KNN Performance

In [31]:
start_time = time.time()
ratings_pred=benchmark_rating_pred(test_movies,5,"knn")
exec_time= time.time() - start_time

In [32]:
print("Performance of KNN : ",get_rmse(ratings_pred),"execution time : ",exec_time)

Performance of KNN :  1.094282606478087 execution time :  0.6530704498291016


In [33]:
ratings_pred

,movie_id,predicted_rating,user_id,actual_rating,squared_diff
0,170.0,3.614029,564.0,4.0,0.148974
1,103731.0,4.388762,652.0,5.0,0.373612
2,173.0,3.185886,151.0,1.0,4.778098
3,5820.0,2.784729,509.0,4.0,1.476884
4,2470.0,3.814848,119.0,4.0,0.034281
5,916.0,4.818961,387.0,4.0,0.670696
6,1632.0,2.700000,165.0,2.5,0.040000
7,1956.0,3.705985,547.0,5.0,1.674474
8,7010.0,3.902032,547.0,4.0,0.009598
9,6936.0,2.836291,654.0,4.5,2.767928
